In [3]:
import pandas as pd

# อ่านไฟล์ CSV
df = pd.read_csv('../dataset-nc/lonlat_rcm_thai_2730.csv', header=None, names=['lon', 'lat'])

# ดูตัวอย่างข้อมูล
print(df.head())

# นับจำนวน unique ของ longitude และ latitude
num_lon = df['lon']
num_lat = df['lat']

# นับจำนวนข้อมูลทั้งหมด
total_points = len(df)

print(f"จำนวนจุดทั้งหมด (grid points): {total_points}")


         lon       lat
0  97.182526  5.599943
1  97.408806  5.599943
2  97.635086  5.599943
3  97.861359  5.599943
4  98.087639  5.599943
จำนวนจุดทั้งหมด (grid points): 2730


In [5]:
# สมมติไฟล์ชื่อว่า 'climate_data.txt' และอยู่ใน path เดียวกับ script
file_path = '../dataset-nc/TH_ensemble_hist_1970_2005_tmean.txt'

with open(file_path, 'r') as f:
    lines = f.readlines()

# นับจำนวนแถว
num_rows = len(lines)

# นับจำนวนคอลัมน์ (สมมติว่าแต่ละค่าคั่นด้วย space)
num_cols = len(lines[0].split())

print(f"จำนวนแถว (rows): {num_rows}")
print(f"จำนวนคอลัมน์ (columns): {num_cols}")


จำนวนแถว (rows): 12897
จำนวนคอลัมน์ (columns): 2730


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# อ่านพิกัด
latlon_df = pd.read_csv('../dataset-nc/lonlat_rcm_thai_2730.csv', header=None, names=['lon', 'lat'])

# โหลดข้อมูล Tmean
data = np.loadtxt('../dataset-nc/TH_ensemble_hist_1970_2005_tmean.txt')

# Transpose
data_T = data.T  # (2730, 12897)

# รวมข้อมูล
df = pd.DataFrame(data_T)
df.insert(0, 'lat', latlon_df['lat'])
df.insert(0, 'lon', latlon_df['lon'])

# บันทึกเป็น CSV
output_path = "../dataset-nc/dataset-test/climate_with_coords_rows.csv"
df.to_csv(output_path, index=False)




In [ ]:
# import pandas as pd
# import numpy as np
# from shapely.geometry import Polygon


# # อ่านไฟล์พิกัด
# df = pd.read_csv('../dataset-nc/lonlat_rcm_thai_2730.csv', header=None, names=['lon', 'lat'])

# # คำนวณค่า unique
# unique_lons = sorted(df['lon'].unique())
# unique_lats = sorted(df['lat'].unique())

# # คำนวณขนาด grid
# delta_lon = np.min(np.diff(unique_lons))
# delta_lat = np.min(np.diff(unique_lats))

# print(f"Δlon = {delta_lon}")
# print(f"Δlat = {delta_lat}")


import pandas as pd
import numpy as np
from shapely.geometry import Polygon, mapping
import json

# ---------------------------------------
# 1. อ่านพิกัด lat/lon ทั้งหมดจาก CSV
# ---------------------------------------
df = pd.read_csv('../dataset-nc/lonlat_rcm_thai_2730.csv', header=None, names=['lon', 'lat'])

# ---------------------------------------
# 2. หาค่าลองจิจูด (lon) และ ละติจูด (lat) ที่ไม่ซ้ำ
# เพื่อดูว่า grid วางอย่างไร และหาความห่าง
# ---------------------------------------
unique_lons = sorted(df['lon'].unique())
unique_lats = sorted(df['lat'].unique())

# ---------------------------------------
# 3. คำนวณระยะห่างของแต่ละช่อง (lon, lat)
# โดยใช้ np.diff แล้วเอาค่าน้อยสุด (เพื่อความปลอดภัยจาก noise)
# ---------------------------------------
delta_lon = np.min(np.diff(unique_lons))
delta_lat = np.min(np.diff(unique_lats))

print(f"lon = {delta_lon}")
print(f"lat = {delta_lat}")

# ---------------------------------------
# 4. ฟังก์ชันสร้างกล่องสี่เหลี่ยมรอบแต่ละจุดศูนย์กลาง
# ใช้ครึ่งหนึ่งของ lon และ lat เพื่อคำนวณมุมทั้ง 4
# ---------------------------------------
def create_grid_box(lon, lat, dlon, dlat):
    half_lon = dlon / 2
    half_lat = dlat / 2
    return Polygon([
        (lon - half_lon, lat - half_lat),
        (lon + half_lon, lat - half_lat),
        (lon + half_lon, lat + half_lat),
        (lon - half_lon, lat + half_lat),
        (lon - half_lon, lat - half_lat)  # ปิด polygon
    ])

# ---------------------------------------
# 5. สร้าง GeoJSON Feature สำหรับแต่ละจุด
# พร้อมฝังค่าพิกัดศูนย์กลางเป็น properties
# ---------------------------------------
features = []
for index, row in df.iterrows():
    lon = row['lon']
    lat = row['lat']
    polygon = create_grid_box(lon, lat, delta_lon, delta_lat)

    feature = {
        "type": "Feature",
        "geometry": mapping(polygon),  # แปลง Polygon เป็น GeoJSON geometry
        "properties": {
            "lon": lon,
            "lat": lat
        }
    }
    features.append(feature)

# ---------------------------------------
# 6. รวมเป็น FeatureCollection
# ---------------------------------------
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# ---------------------------------------
# 7. บันทึกเป็นไฟล์ GeoJSON
# ---------------------------------------
with open("../Geo-data/thai_grid_boxes.json", "w") as f:
    json.dump(geojson, f, indent=4)

print("GeoJSON finish")


lon = 0.22627199999999448
lat = 0.211743000000002
GeoJSON finish
